In [7]:
%load_ext autoreload
%autoreload 2
# System modules
import os
import warnings
import pickle
import glob
import sys
import logging

# External modules
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import simplekml

# Internal modules
import pyclamster
from pyclamster.clustering.preprocess import LCN
from pyclamster.functions import rbDetection
from pyclamster.positioning import Projection

warnings.catch_warnings()
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
FILE_DIR = %pwd
BASE_DIR = os.path.dirname(FILE_DIR)

image_directory = os.path.join(BASE_DIR, "examples", "images", "lex")
trained_models = os.path.join(BASE_DIR, "data")
plot_dir = os.path.join(BASE_DIR, 'plots')

kml_file = simplekml.Kml()


In [3]:
predictor = pickle.load(open(os.path.join(trained_models, "kmeans.pk"), "rb"))

cams = []
cams.append(pickle.load(open(os.path.join(trained_models, 'sessions', 'FE3_session.pk'), mode='rb')))
cams.append(pickle.load(open(os.path.join(trained_models, 'sessions', 'FE4_session.pk'), mode='rb')))
cams[0].set_images(os.path.join(image_directory, 'cam3'))
cams[1].set_images(os.path.join(image_directory, 'cam4'))
matching = pyclamster.matching.Matching()

In [4]:
times = {'3': [], '4': []}
for img3 in cams[0]:
    img3.loadTimefromfilename('FE3_Image_%Y%m%d_%H%M%S_UTCp1.jpg')
    times['3'].append(img3.time)

for img4 in cams[1]:
    img4.loadTimefromfilename('FE4_Image_%Y%m%d_%H%M%S_UTCp1.jpg')
    times['4'].append(img4.time)

key_pair = [(k, times['4'].index(t)) for k, t in enumerate(times['3']) if t in times['4']]

In [ ]:
t = 0
for keys in key_pair:
    i = 0
    clouds = []
    for k in keys:
        img = cams[i][k]
        if i==0:
            img.loadTimefromfilename('FE3_Image_%Y%m%d_%H%M%S_UTCp1.jpg')
        else:
            img.loadTimefromfilename('FE4_Image_%Y%m%d_%H%M%S_UTCp1.jpg')
        scipy.misc.imsave(
            os.path.join(plot_dir, "rectified_{0:d}_{1:d}.png".format(i, k)),
            img.image)
        image_lcn = pyclamster.Image(img)
        image_lcn.data = LCN(size=(50, 50, 3), scale=False).fit_transform(
            image_lcn.data / 256)
        w, h, _ = original_shape = image_lcn.data.shape
        raw_image_lcn = rbDetection(image_lcn.data).reshape((w * h, -1))
        label = predictor.predict(raw_image_lcn)
        label.reshape((w, h), replace=True)
        scipy.misc.imsave(
            os.path.join(plot_dir, "lables_kmean_{0:d}_{1:d}.png".format(i, k)),
            label.labels)
        masks = label.getMaskStore()
        cloud_mask_num = [1] # cloud - sky choose right number (0 or 1)
        masks.denoise(cloud_mask_num,
                      5000)
        cloud_labels_object, numLabels = masks.labelMask(cloud_mask_num)
        scipy.misc.imsave(
            os.path.join(plot_dir, "labl"
                                   "es_used_{0:d}_{1:d}.png".format(i, k)),
            cloud_labels_object.labels)
        cloud_store = cloud_labels_object.getMaskStore()
        cloud_lables = [l + 1 for l in range(numLabels)]
        clouds.append([cloud_store.getCloud(img, [k, ]) for k in cloud_lables])
        j = 0
        #print(clouds[i])
        for cloud in clouds[i]:
            scipy.misc.imsave(
                os.path.join(plot_dir, 'template_cloud_{0:d}_{1:d}_{2:d}.png'.format(i, k, j)),
                cloud.image.data)
            j += 1
        print('finished image {0:d} of camera {1:d}'.format(k, i))
        i += 1
        print(type(clouds[0][0].image.data))
    if not(not clouds[0] or not clouds[1]):
        matching_result, _ = matching.matching(clouds[0], clouds[1], min_match_prob=0.5)
        t = 0
        for result in matching_result:
            fig = plt.figure()
            ax = plt.subplot(1,3,1)
            ax.axis('off')
            ax.imshow(result[1].clouds[0].image.data)
            ax = plt.subplot(1,3,2)
            ax.axis('off')
            ax.imshow(result[0].prop_map, interpolation='nearest')
            ax = plt.subplot(1,3,3)
            ax.axis('off')
            ax.imshow(result[1].clouds[1].image.data)
            plt.tight_layout()
            plt.savefig(os.path.join(plot_dir, 'matching_{0:s}_{1:d}.png'.format(str(keys), t)))
            spatial_cloud = result[1]
            spatial_cloud.calc_overlapping()
            spatial_cloud.calc_position()
            spatial_cloud.write2kml(kml_file)
            kml_file.save(os.path.join(trained_models, 'clouds.kml'))
            print(Projection().xy2lonlat(spatial_cloud.positions.x, spatial_cloud.positions.y))
            t+=1
    print('finished image pair {0:s}'.format(str(keys)))


finished image 0 of camera 0
<class 'numpy.ndarray'>
finished image 0 of camera 1
<class 'numpy.ndarray'>
(masked_array(data =
 [[ 54.4800559   54.48063329  54.48123508 ...,  54.58839175  54.58804686
   54.58767972]
 [ 54.47997995  54.48054873  54.48114212 ...,  54.5875492   54.58719049
   54.58680951]
 [ 54.47990716  54.48046719  54.48105206 ...,  54.58669811  54.58632547
   54.58593055]
 ..., 
 [ 54.54039504  54.53912189  54.53784389 ...,  54.39897312  54.39760172
   54.39623271]
 [ 54.53861496  54.53731661  54.53601233 ...,  54.39989895  54.39851928
   54.39714202]
 [ 54.53676944  54.53544559  54.53411482 ...,  54.40085372  54.39946594
   54.39808059]],
             mask =
 False,
       fill_value = 1e+20)
, masked_array(data =
 [[ 11.36382029  11.36544527  11.36705548 ...,  11.34473341  11.34332943
   11.34192654]
 [ 11.36238317  11.36401332  11.3656288  ...,  11.34320074  11.34179687
   11.34039432]
 [ 11.36094385  11.36257915  11.36419991 ...,  11.34166778  11.34026416
   11.338

In [ ]:
kml_file.save(os.path.join(trained_models, 'clouds.kml'))